In [1]:
import multiprocessing as mp
from pathlib import Path

import numpy as np
import pandas as pd
import wfdb
import wfdb.io
from tqdm import tqdm

----

In [2]:
master_header = wfdb.io.rdheader('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p04/p044083/p044083-2112-05-04-19-50', rd_segments=True)
master_header

In [3]:
segment_header = wfdb.io.rdheader('/home/thulio/Downloads/mimic/physionet.org/files/mimic3wdb-matched/1.0/p04/p044083/3343009_0001')
segment_header

In [4]:
master_header.sig_name

['II', 'AVR', 'V', 'MCL', 'RESP', 'PLETH', 'ABP']

In [5]:
master_header.contained_ranges('II')

[(0, 7535000),
 (7535658, 15344033),
 (15345033, 16154283),
 (16155283, 16363783),
 (16364658, 16633533),
 (16634283, 17151158),
 (17152033, 17368908),
 (17369783, 20342033)]

In [6]:
master_header.contained_combined_ranges(['II', 'RESP', 'PLETH', 'ABP'])

[(38875, 6203500), (6281500, 7535000), (7535658, 9734908)]

In [7]:
# total_samples = sum([end - start for start, end in master_header.contained_combined_ranges(relevant_signals)])
# total_minutes = total_samples / 125 / 60
# print(f'{total_minutes:.2f} min')

In [8]:
req_signals = ['II', 'PLETH', 'RESP']
min_time = 5 * 60
good_times = []
good_segments = []
for seg in master_header.segments:
    if seg is not None:
        seg: wfdb.io.Record
        if seg.sig_len > seg.fs * min_time:
            if all([sig in seg.sig_name for sig in req_signals]):
                good_times.append(seg.sig_len / seg.fs)
                good_segments.append(seg.record_name)
print(f'{len(good_segments)} segments, {sum(good_times)/60:.2f} minutes')

14 segments, 2684.52 minutes


In [9]:
good_segments

['3314767_0005',
 '3314767_0007',
 '3314767_0010',
 '3314767_0013',
 '3314767_0017',
 '3314767_0020',
 '3314767_0021',
 '3314767_0023',
 '3314767_0026',
 '3314767_0029',
 '3314767_0032',
 '3314767_0035',
 '3314767_0038',
 '3314767_0041']